<center>
<a href="http://www.insa-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo-insa.jpg" style="float:left; max-width: 120px; display: inline" alt="INSA"/></a> 

<a href="http://wikistat.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/wikistat.jpg" style="float:right; max-width: 250px; display: inline"  alt="Wikistat"/></a>

</center>

# # [Statistical learning scenarios](https://github.com/wikistat/Apprentissage)

#  Statistical Adaptation of an Ozone Peak Forecasting Model with <a href="https://www.python.org/"><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/f/f8/Python_logo_and_wordmark.svg/390px-Python_logo_and_wordmark.svg.png" style="max-width: 120px; display: inline" alt="Python"/></a> avec <a href="http://scikit-learn.org/stable/#"><img src="http://scikit-learn.org/stable/_static/scikit-learn-logo-small.png" style="max-width: 100px; display: inline" alt="Scikit-learn"/></a>


**Abstract**: Exploration and modeling of climate data using Python and the [Scikit-learn library](http://scikit-learn.org/stable/#). The objective is to forecast for the following day a possible exceedance of an ozone concentration threshold from a deterministic forecast on a coarse mesh and local climatic variables. Estimation by different methods: [logistic regression](http://wikistat.fr/pdf/st-m-app-rlogit.pdf), [k nearest neighbors](http://wikistat.fr/pdf/st-m-app-add.pdf), [decision tree](http://wikistat.fr/pdf/st-m-app-cart.pdf), [model aggregation](http://wikistat.fr/pdf/st-m-app-agreg.pdf), [SVM](http://wikistat.fr/pdf/st-m-app-svm.pdf). Comparison of [prediction errors](http://wikistat.fr/pdf/st-m-app-risque-estim.pdf) on a test sample, then ROC curves. Iteration on several test samples to analyze forecast error distribution. This notebook complements the [study done with R](http://www.math.univ-toulouse.fr/~besse/Wikistat/Notebooks/Notebook-R-Ozone.html) to compare the two approaches.



**Warning** 

* This tutorial complements [the one in R](https://github.com/wikistat/MLTraining/tree/master/Notebooks/Ozone/Apprent-R-Ozone.ipynb) to compare the respective performances of the two environments: completeness of results and code efficiency. Explanations are more brief in this tutorial, which is usually run *after* or in parallel with the R tutorial.  
* As with R, it is *divided into 5 tutorial sessions* *synchronized* with the machine learning course. 
* Not all options have been tested, and some are included as **exercises**.

## Introduction

The objective, on these data, is to improve the deterministic forecast (MOCAGE), calculated by the services of MétéoFrance, of the ozone concentration in some sampling stations.  It is a problem of "statistical adaptation" of a local forecast of too large scale models with the help of other variables also managed by MétéoFrance, but on a smaller scale (temperature, wind strength...). This is a first way to design *IA hybrid* between a deterministic model and a machine learning algorithm. More precisely, two variables can be predicted: either the quantitative concentration of ozone, or the (qualitative) exceedance of a certain threshold set at $150$. In each case, two approaches are considered : either predict the *quantitative concentration* and then deduce the possible exceedance or directly predict the *exceedance*. In the first case, it is first a *regression* while in the second it is a two-class *discrimination* or logistic regression problem. 

The question is therefore: what are the best methods and strategies to predict the next day's ozone concentration on the one hand and the occurrence of a pollution peak on the other hand.

We propose to test different methods: [logistic regression](http://wikistat.fr/pdf/st-m-app-rlogit.pdf), [discriminant analysis](http://wikistat.fr/pdf/st-m-app-add.pdf), [neural network](http://wikistat.fr/pdf/st-m-app-rn.pdf), [decision tree](http://wikistat.fr/pdf/st-m-app-cart.pdf), [tree aggregation](http://wikistat.fr/pdf/st-m-app-agreg.pdf) (random forest), [SVM](http://wikistat.fr/pdf/st-m-app-svm.pdf).  The final objective is the comparison of these methods in order to determine the most efficient one to answer the forecasting problem. This requires the implementation of a very strict protocol to ensure a minimum of objectivity for this comparison.

**Subsidiary question** When to choose R or Python? Python leads to results (conclusions) identical to those of R, less complete for their interpretation, but faster. These are the main differences between R for "statisticians" and Python for "computer scientists": you lose in interpretability but gain in execution speed.

# <FONT COLOR="Red">Episode 1 : Descriptive statistics and linear models </font>

## Data munging


The data were extracted and formatted by the relevant department of Météo France. They are described by the following variables :

* **JOUR**: type of day; holiday (1) or not (0) ;
* **O3obs**: ozone concentration actually observed the next day at 5 pm local time, often corresponding to the maximum pollution observed;
* **MOCAGE**: forecast of this pollution obtained by a deterministic fluid mechanics model (Navier and Stockes equation);
* **TEMPE**: temperature forecast by Météo France for the next day at 5 pm;
* **RMH2O** : humidity ratio;
* **NO2** : nitrogen dioxide concentration;
* **NO** : nitrogen monoxide concentration;
* **STATION**: observation location: Aix-en-Provence, Rambouillet, Munchhausen, Cadarache and Plan de Cuques;
* **WindMOD**: wind strength;
* **WindANG**: wind direction. 

These are "clean" data, without missing values, well coded and small in size. They are therefore primarily of an educational nature, as they allow us to use and compare all the regression and supervised classification approaches.

Here, we have chosen to read the data with the `pandas` library to benefit from the DataFrame class. This is not necessary for the forecasting objective, as the categorical variables thus constructed cannot be used to interpret the models obtained in `scikit-learn`, which does not recognize the DataFrame class.

**Caution**: Even if the data are of good quality, a preliminary exploratory study is always necessary to become familiar with the data and prepare them for the modeling phase.


In [ ]:
import pandas as pd
import numpy as np

path=""
ozone=pd.read_csv(path+"depSeuil.dat",sep=",",header=0)

ozone.head()

Ce qui suit permet d'affecter le bon type aux variables.

In [ ]:
ozone["STATION"]=pd.Categorical(ozone["STATION"],ordered=False)
ozone["JOUR"]=pd.Categorical(ozone["JOUR"],ordered=False)
ozone["O3obs"]=pd.DataFrame(ozone["O3obs"], dtype=float)
ozone.dtypes

In [ ]:
ozone.describe()

## Basic exploration

Even if the data have no particular flaws, a preliminary exploratory study is essential to ensure that they are consistent, to suggest possible transformations and to analyze correlation structures or, more generally, links between variables, groups of individuals or observations.


### Unidimensional statistics

In [ ]:
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt
ozone["O3obs"].hist()
plt.show()

In [ ]:
ozone["MOCAGE"].hist()
plt.show()

**Exercice** Traitez ainsi toutes les variables. Ceci suggère des transformations pour une meilleure utilisation des modèles linéaires.  

In [ ]:
from math import sqrt, log
ozone["SRMH2O"]=ozone["RMH2O"].map(lambda x: sqrt(x))
ozone["LNO2"]=ozone["NO2"].map(lambda x: log(x))
ozone["LNO"]=ozone["NO"].map(lambda x: log(x))
del ozone["RMH2O"]
del ozone["NO2"]
del ozone["NO"]

**Exercise** Check the appropriateness of these transformations (histograms of the new variables).

Remove the initial variables and construct the "threshold exceeded" variable below to obtain the file that will actually be used.

In [ ]:
ozone["DepSeuil"]=ozone["O3obs"].map(lambda x: x > 150)
ozone.head()

### Multidimensional exploration

In [ ]:
# scatter plot matrix for  quantitative variables
from pandas.plotting import scatter_matrix
scatter_matrix(ozone[["O3obs","MOCAGE","TEMPE","VentMOD","VentANG","SRMH2O","LNO2","LNO"]], alpha=0.2, 
               figsize=(15, 15), diagonal='kde')
plt.show()

**Question** Comment on the relationships between the variables taken 2 by 2.

   ### Principal Components Analysis

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
# scaling of the  variables
X=scale(ozone[["MOCAGE","TEMPE","VentMOD","VentANG","SRMH2O","LNO2","LNO"]])

All classical numerical results are provided by the [PCA implementation](http://scikit-learn.org/stable/modules/decomposition.html) with `scikit-learn`, but more effort is needed to build the usual graphs usually automatically produced by dedicated libraries such as R's [FactoMineR](http://factominer.free.fr/) and [factoExtra](https://cran.r-project.org/web/packages/factoextra/index.html).

The following commands allow you to perform a principal component analysis on quantitative variables only. In addition, the variable to be modeled (O3obs, observed concentration) is not used.

In [ ]:
pca = PCA()
## Estimation, computation of the principal components
C = pca.fit(X).transform(X)

plt.plot(pca.explained_variance_ratio_)
plt.show()

In [ ]:
## distribution of the  principal components
plt.boxplot(C[:,0:20])
plt.show()

**Question** Comment on these results: which dimension was chosen?

**Question** Presence of atypical values.


In [ ]:
## Représentation of the individuals
plt.figure(figsize=(5,5))
for i, j, nom in zip(C[:,0], C[:,1], ozone["DepSeuil"]):
    color = "red" if nom  else "blue"
    plt.plot(i, j, "o",color=color)
plt.axis((-4,6,-4,6))  
plt.show()

In [ ]:
## coordinates and representation of the variables
coord1=pca.components_[0]*np.sqrt(pca.explained_variance_[0])
coord2=pca.components_[1]*np.sqrt(pca.explained_variance_[1])
fig = plt.figure(figsize=(5,5))
ax = fig.add_subplot(1, 1, 1)
for i, j, nom in zip(coord1,coord2, ozone[["MOCAGE","TEMPE","VentMOD",
                                           "VentANG","SRMH2O","LNO2","LNO"]].columns):
    plt.text(i, j, nom)
    plt.arrow(0,0,i,j,color='black')
plt.axis((-1.2,1.2,-1.2,1.2))
# cercle
c=plt.Circle((0,0), radius=1, color='gray', fill=False)
ax.add_patch(c)
plt.show()

**Question** Comment on the correlation structure of the variables.

**Question** The aim is to define a surface separating the two classes. Does a linear discriminaiton (hyperplane) seem possible?   

## Comparison Protocol

### Strategy


The search for a better prediction method follows the following protocol.

1. Preliminary uni- and multidimensional descriptive steps to identify inconsistencies, non-significant or exotic distribution variables, irrelevant or atypical individuals... and to study the data structures. It can also be the long step of building specific variables, attributes or *features* of the data. 
2. Randomly draw a *test* sample that will only be used in the *last step* of comparing methods.
3. The remaining part is the *training* sample for estimating the parameters of the models.
4. For each of the methods, optimize the complexity of the models by minimizing an "unbiased" estimate of the prediction error, e.g., by [*cross validation*](http://wikistat.fr/pdf/st-m-app-risque.pdf):
    - Variables and interactions to be considered in linear or logistic regression;
    - variables and method for discriminant analysis;
    - number of leaves in the regression or classification tree;
    - architecture (number of neurons, penalization) of the perceptron;
    - aggregation algorithms, 
    - kernel and penalization of SVMs.
5.  Comparison of predictive qualities based on the misclassification rate for the  test sample.



**Notes**
* In the case of a relatively "small" sample, it is recommended to iterate the learning/test splitting procedure ([cross-validation *Monte Carlo*](http://wikistat.fr/pdf/st-m-app-risque-estim.pdf)), in order to reduce the (average) variance of prediction error estimates.
* *Caution*: do not "cheat" by modifying the model obtained in the previous step in order to improve the result on the test sample!
* The criterion used depends on the problem: squared error, misclassification rate, AUC (area under the ROC curve), Pierce index, *log loss function*...
* The "choice" of the best method can be replaced by a prediction combination, as is often the case with the "winning" but cumbersome soutions on the [kaggle site](https://www.kaggle.com/competitions).



### Extraction of the samples

Data transformation for learning. 

**Question** Why are categorical variables transformed into indicator bundles or *dummy variables*?

**Question** Why is the data frame type transformed into a matrix? 

In [ ]:
ozone.head()

In [ ]:
# Explanatory Variables
ozoneDum=pd.get_dummies(ozone[["JOUR","STATION"]])
del ozoneDum["JOUR_0"]
ozoneQuant=ozone[["MOCAGE","TEMPE","VentMOD","VentANG","SRMH2O","LNO2","LNO"]]
dfC=pd.concat([ozoneDum,ozoneQuant],axis=1)
dfC.head()

In [ ]:
# binary variable  to predict
Yb=ozone["DepSeuil"].map(lambda x: int(x))
# Real variable to predict
Yr=ozone["O3obs"]

In [ ]:
Yr.hist()
plt.show()

Extraction of training and test samples for both types of model. As the generator is initialized identically, the same samples are used in both cases.

In [ ]:
from sklearn.model_selection import train_test_split  
X_train,X_test,Yb_train,Yb_test=train_test_split(dfC,Yb,test_size=200,random_state=11)
X_train,X_test,Yr_train,Yr_test=train_test_split(dfC,Yr,test_size=200,random_state=11)

The next step is data standardization or normalization. The variables are divided by their standard deviation. This is not useful in the case of an elementary linear model, as the solution is identical, but it is essential for many other non-linear methods (SVMs, neural networks, penalized models). This step is therefore systematically carried out to avoid any problems. 

**Please note**: the same parameters (means, standard deviations) estimated on the training sample are used to normalize the test sample.   

In [ ]:
from sklearn.preprocessing import StandardScaler  
# The neural network algorithm may require normalization of the 
# of the explanatory variables with the commands below
scaler = StandardScaler()  
scaler.fit(X_train)  
Xr_train = scaler.transform(X_train)  
# Same transformation for the test
Xr_test = scaler.transform(X_test)

## Prediction by linear models

Linear and generalized linear model functions are limited in [Scikit-learn](http://scikit-learn.org/stable/supervised_learning.html#supervised-learning) and without detailed numerical (test) outputs. It is preferable to use another library [StatsModels](http://statsmodels.sourceforge.net/stable/examples/notebooks/generated/glm.html) whose outputs are inspired by those of R. In both cases, the classical strategies (forward, backward, stepwise) of variable selection by optimization of a criterion (Cp, AIC, BIC) do not seem to be available, even if AIC and BIC are present in `scikit-learn`, and the DataFrame type (package `pandas`) is not recognized.

The most efficient way to proceed is therefore to introduce a [Lasso penalization](http://wikistat.fr/pdf/st-m-app-select.pdf) to carry out variable selection, or rather the selection of quantitative variables and indicators of the modalities of qualitative ones, but without detailed analysis of interactions as is possible with R.

**Question** What other type of penalization is also used in regression?

**Question** Which method combines the two?

### Preliminary comparison  with  MOCAGE

For comparison, we plot the concentration prediction for the test sample using only the value of the *Mocage* model, as well as the residuals of this model as a function of the predicted value (Mocage).

In [ ]:
plt.plot(X_train["MOCAGE"],Yr_train,"o")
plt.xlabel("Mocage")
plt.ylabel("O3 observee")
plt.show()

In [ ]:
from sklearn.metrics import r2_score
print("R2=",r2_score(Yr_train,X_train["MOCAGE"]))

In [ ]:
plt.plot(X_test["MOCAGE"],Yr_test,"o")
plt.xlabel("Mocage")
plt.ylabel("O3 observee")
plt.show()

In [ ]:
plt.plot(X_test["MOCAGE"],X_test["MOCAGE"]-Yr_test,"o")
plt.xlabel("Mocage")
plt.ylabel("Residus")
plt.show()

**Question** Comment on the quality of these residuals.

In [ ]:
# Mean squared error
from sklearn.metrics import mean_squared_error
print("MSE=",mean_squared_error(X_test["MOCAGE"],Yr_test))

In [ ]:

from sklearn.metrics import r2_score
print("R2=",r2_score(Yr_test,X_test["MOCAGE"]))

### Linear regression model

Compare this deterministic forecast (Navier and Stockes equation) with the most elementary statistical adaptation. This is a regression with model choice by regularization with a Lasso penalty. 

**Question** What is the default value of the Lasso penalty parameter?

In [ ]:
from sklearn import linear_model
regLasso = linear_model.Lasso()
regLasso.fit(Xr_train,Yr_train)
prev=regLasso.predict(Xr_test)
print("MSE=",mean_squared_error(Yr_test,prev))

In [ ]:
from sklearn.metrics import r2_score
print("R2=",r2_score(Yr_test,prev))

The Lasso penalty parameter is optimized by cross-validation.

In [ ]:
from sklearn.model_selection import GridSearchCV
# Grid of values for the parameter  alpha to optimize
param=[{"alpha":[0.05,0.1,0.2,0.3,0.4,0.5,1]}]
regLasso = GridSearchCV(linear_model.Lasso(), param,cv=5,n_jobs=-1)
regLassOpt=regLasso.fit(Xr_train, Yr_train)
# optimal parameter
regLassOpt.best_params_["alpha"]
print("Meilleur R2 = %f, Meilleur paramètre = %s" % (regLassOpt.best_score_,regLassOpt.best_params_))

**Question** Which cross-validation is performed?

Obtain a forecast with the optimal value of `alpha` then calculate and plot residuals.

In [ ]:
prev=regLassOpt.predict(Xr_test)
print("MSE=",mean_squared_error(prev,Yr_test))
print("R2=",r2_score(Yr_test,prev))

In [ ]:
plt.plot(prev,Yr_test,"o")
plt.xlabel(u"O3 Prédite")
plt.ylabel("O3 observee")
plt.show()

In [ ]:
plt.plot(prev,Yr_test-prev,"o")
plt.xlabel(u"Prédites")
plt.ylabel(u"Résidus")
plt.hlines(0,40,220)
plt.show()

**Question** Compare these residuals with the previous ones (mocage) and note the improvement. 

**Question** Comment on the shape of the cloud and therefore the validity of the model. 

Interpretation requires knowing the values of the model's coefficients, whereas the `regLassOpt` object from `GridSearchCV` does not retain the estimated parameters. It must therefore be re-estimated with the optimal value of the penalty parameter if we wish to display these coefficients.

In [ ]:
# Coefficients
regLasso=linear_model.Lasso(alpha=regLassOpt.best_params_['alpha'])
model_lasso=regLasso.fit(Xr_train,Yr_train)
model_lasso.coef_

In [ ]:
coef = pd.Series(model_lasso.coef_, index = X_train.columns)
print("Lasso conserve " + str(sum(coef != 0)) + 
      " variables et en supprime " +  str(sum(coef == 0)))

In [ ]:
imp_coef = coef.sort_values()
plt.rcParams['figure.figsize'] = (8.0, 10.0)
imp_coef.plot(kind = "barh")
plt.title(u"Coefficients du modèle lasso")

**Question** Note the consequences of penalization; interpret the effect of each variable on ozone concentration.

This is where the python library comes up short. You'd have to build "by hand" or use the `Statsmodels` library to display test statistics and p-values. Even with these additions, there is no provision for taking interactions and their selection into account. Furthermore, interpretation is complicated by the fact that each qualitative variable is broken down into packets of indicators. This is still understandable with a small number of variables, but quickly becomes unworkable.

The following graph shows the good and bad forecasts for exceeding the legal threshold, here set at $ 150 \mu g $.

In [ ]:
plt.plot(prev,Yr_test,"o")
plt.xlabel(u"Valeurs prédites")
plt.ylabel(u"O3 observée")
plt.hlines(150,50,300)
plt.vlines(150,0,300)
plt.show()

In [ ]:
#Confusion matrix
table=pd.crosstab(prev>150,Yr_test>150)
print(table)

**Question** Observe the asymmetry of this matrix. What is it at least partly due to?

The `lassoCV` uses a *coordinate descent* algorithm, with no derivative calculation since the *l1* norm is not derivable, while `lassoLarsCV` is based on the *least angle regression* algorithm. These functions also plot *regularization paths*. Here's an example of `lassoCV`, which offers more options.

In [ ]:
from sklearn.linear_model import LassoCV, LassoLarsCV
model = LassoCV(cv=5, alphas=np.array(range(1,50,1))/20.,n_jobs=-1,random_state=13).fit(Xr_train,Yr_train)
m_log_alphas = -np.log10(model.alphas_)

plt.figure()
# ymin, ymax = 2300, 3800
plt.plot(m_log_alphas, model.mse_path_, ':')
plt.plot(m_log_alphas, model.mse_path_.mean(axis=-1), 'k',
         label='MSE moyen', linewidth=2)
plt.axvline(-np.log10(model.alpha_), linestyle='--', color='k',
            label='alpha: optimal par VC')

plt.legend()

plt.xlabel('-log(alpha)')
plt.ylabel('MSE')
plt.title('MSE for each validation: coordinate descent ')
plt.show()

**Question** Check that this is the same optimal value as the one you found previously.

Drawing of regularization paths.

In [ ]:
from itertools import cycle

from sklearn.linear_model import lasso_path
alphas_lasso, coefs_lasso, _ = lasso_path(Xr_train,Yr_train, alphas=np.array(range(1,50,1))/20.,)


plt.figure()
ax = plt.gca()

styles = cycle(['-', '--', '-.', ':'])

neg_log_alphas_lasso = -np.log10(alphas_lasso)
for coef_l, s in zip(coefs_lasso, styles):
    l1 = plt.plot(neg_log_alphas_lasso, coef_l, linestyle=s,c='b')
plt.xlabel('-Log(alpha)')
plt.ylabel('Coefficients')
plt.show()

### Logistic regression or binomial model

The same approach is used, but directly modeling the binary variable of whether or not the threshold is exceeded. This is a logistic regression, with a Lasso penalty to select the variables.

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
# Optimisation of the penalisation parameter
# grid of values
param=[{"C":[1,1.2,1.5,1.7,2,3,4]}]
logit = GridSearchCV(LogisticRegression(penalty="l1",solver="liblinear"), param,cv=5,n_jobs=-1)
logitOpt=logit.fit(Xr_train, Yb_train) 
# optimal parameter
logitOpt.best_params_["C"]
print("Meilleur score = %f, Meilleur paramètre = %s" % (1.-logitOpt.best_score_,logitOpt.best_params_))

In [ ]:
# Error on the test sample
1-logitOpt.score(Xr_test, Yb_test)

The "optimal" model obtained is used to predict the test sample and thus estimate, without bias, a prediction error. 

The confusion matrix cross-references predicted threshold violations with those actually observed. 

In [ ]:
# Prediction
y_chap = logitOpt.predict(Xr_test)
# confusion matrix
table=pd.crosstab(y_chap,Yb_test)
print(table)

Interpretation of the model is based on the values of the coefficients, with the same difficulties or restrictions as for regression. Please note that `GridSearch` does not retain the coefficients, which must be re-estimated.

In [ ]:
# Coefficients
logitLasso=LogisticRegression(penalty="l1",C=logitOpt.best_params_['C'],
                              solver="liblinear")
logitCoef=logitLasso.fit(Xr_train,Yb_train).coef_
print(logitCoef[0])

In [ ]:
coef = pd.Series(logitCoef[0], index = X_train.columns)
print("Lasso conserve " + str(sum(coef != 0)) + 
      " variables et en supprime " +  str(sum(coef == 0)))

In [ ]:
imp_coef = coef.sort_values()
plt.rcParams['figure.figsize'] = (6.0, 6.0)
imp_coef.plot(kind = "barh")
plt.title(u"Coefficients du modèle lasso")

**Question** Interpret the effect of the variables selected.

In [ ]:
from sklearn.metrics import roc_curve
probas_ = LogisticRegression(penalty="l1", solver="liblinear",
                    C=logitOpt.best_params_['C']).fit(X_train, Yb_train).predict_proba(X_test)
fpr, tpr, thresholds = roc_curve(Yb_test, probas_[:,1])
plt.plot(fpr, tpr, lw=1)
plt.xlabel('Taux de faux positifs')
plt.ylabel('Taux de vrais positifs')
plt.show()

**Question** Comment on the ROC curve regarding the choice of threshold value.

# <FONT COLOR="Red">Episode 2 :  KNN, SVM </font>

### K nearest neighbours

Here's a case study of [non-parametric discriminant analyses](http://scikit-learn.org/stable/modules/neighbors.html). Linear and quadratic [parametric discriminant analyses](http://scikit-learn.org/stable/modules/lda_qda.html) (Gaussian) are also available in `scikit-learn`, but are left as an exercise.

The complexity parameter $K$ is optimized on a predefined grid, minimizing the estimated error by cross-validation; `scikit-learn` offers a number of cross-validation options.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
# Optimisation of k
# grid of values
param_grid=[{"n_neighbors":list(range(1,15))}]
knn=GridSearchCV(KNeighborsClassifier(),param_grid,cv=5,n_jobs=-1)
knnOpt=knn.fit(Xr_train, Yb_train)  
# optimal parameter
knnOpt.best_params_["n_neighbors"]
print("Meilleur score = %f, Meilleur paramètre = %s" % (1.-knnOpt.best_score_,knnOpt.best_params_))

In [ ]:
# Estimation  of the prediction error on the test sample
1-knnOpt.score(Xr_test,Yb_test)

In [ ]:
# Prediction on the test sample
y_chap = knnOpt.predict(Xr_test)
# confusion matrix
table=pd.crosstab(y_chap,Yb_test)
print(table)

**Exercise** Complete the results using the function [KNeighborsRegressor](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html) to model the concentration; optimize $K$, calculate the test sample prediction, plot the residuals, calculate the MSE on the test sample.

### [*Support Vector Machine*](http://wikistat.fr/pdf/st-m-app-svm.pdf)

Many parameters are associated with this method. The list can be consulted in the online [documentation](http://scikit-learn.org/stable/modules/svm.html).

Penalization optimization (parameter C) is sought on a grid by cross-validation.

Note: it would also be necessary to optimize the value of the *gamma* coefficient linked to the Gaussian kernel ("standard deviation").

It is often necessary to normalize data before operating SVMs.

In [ ]:
from sklearn.svm import SVC
param=[{"C":[0.4,0.5,0.6,0.8,1,1.4]}]
svm= GridSearchCV(SVC(),param,cv=10,n_jobs=-1)
svmOpt=svm.fit(Xr_train, Yb_train)
# optimal parameter
print("Best score = %f, Best parameter = %s" % (1. - svmOpt.best_score_,svmOpt.best_params_))

In [ ]:
# Error on the test sample
1-svmOpt.score(Xr_test,Yb_test)

In [ ]:
# Prediction on the  test sample
y_chap = svmOpt.predict(Xr_test)
# Confusion matrix
table=pd.crosstab(y_chap,Yb_test)
print(table)

**Exercise** As before, replace the SVC function with the [SVR](http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html#sklearn.svm.SVR) regression function. Optimize the parameter, calculate the forecast, the residuals and the MSE.

# <FONT COLOR="Red">Episode 3 :  CART, Random Forests  </font>



### [Binary decision trees (CART)](http://wikistat.fr/pdf/st-m-app-cart.pdf)


[Binary classification  and regression trees](http://scikit-learn.org/stable/modules/tree.html) are well implemented in scikit-learn, but their pruning is inadequate. It's not complexity that is penalized, and therefore precisely the number of leaves that is optimized, but the overall depth of the tree, at the risk of pruning, at a given depth, important leaves or keeping ambiguous leaves.

As before, cross-validation is used to optimize the parameter on a grid.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
# Optimisation of the tree depth
param=[{"max_depth":list(range(2,10))}]
tree= GridSearchCV(DecisionTreeClassifier(),param,cv=10,n_jobs=-1)
treeOpt=tree.fit(Xr_train, Yb_train)
# optimal paramter
print("Best score = %f, Best parameter = %s" % (1. - treeOpt.best_score_,treeOpt.best_params_))

In [ ]:
# Estimation of the prediction error
1-treeOpt.score(Xr_test,Yb_test)

In [ ]:

y_chap = treeOpt.predict(Xr_test)
# confusion matrix
table=pd.crosstab(y_chap,Yb_test)
print(table)

We plot the obtained tree with  `plot_tree()`

In [ ]:

from sklearn.tree import plot_tree
import matplotlib.pyplot as plt
treeG=DecisionTreeClassifier(max_depth=treeOpt.best_params_['max_depth'])
treeG.fit(Xr_train,Yb_train)
plot_tree(treeG,feature_names=dfC.columns.tolist());
plt.show()


**Question** What about the interpretation of the tree? Compare the roles of the variables with the logit model.

**Exercise** Complete the results using the [DecisionTreeRegressor](http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html) function to model concentration; optimize depth, calculate test sample prediction, plot residuals, calculate MSE on test sample.

## Random forests

R's *randomForest* library uses the historic program developed by [Breiman and Cutler](https://www.stat.berkeley.edu/~breiman/RandomForests/cc_software.htm) (2001) and interfaced by [Liaw and Wiener](https://cran.r-project.org/web/packages/randomForest/randomForest.pdf). This interface is still being updated, but it's not certain that the original program has continued to evolve since 2004. For large sample sizes, a few thousand, this implementation reaches prohibitive execution times (cf. this [example](https://github.com/wikistat/Ateliers-Big-Data/blob/master/2-MNIST/Atelier-MNIST-R.ipynb)), unlike the Python implementation whose memory management and parallelization capacity have been finely optimized by [Louppe et al.](http://fr.slideshare.net/glouppe/accelerating-random-forests-in-scikitlearn) (2014). 

Like boosting, two forest functions are offered in `scikit-learn`; one for regression and one for classification, as well as a "more random" version. Compared with the original version of R, there are fewer options, but the basic use is very similar, with the same set of parameters.

**Question** Identify the parameters and default values.

In [ ]:
from sklearn.ensemble import RandomForestClassifier 
# definition of the parameters
forest = RandomForestClassifier(n_estimators=500, 
   criterion='gini', max_depth=None,
   min_samples_split=2, min_samples_leaf=1, 
   max_features='auto', max_leaf_nodes=None,
   bootstrap=True, oob_score=True)
# Learning
rfFit = forest.fit(Xr_train,Yb_train)
print(1-rfFit.oob_score_)

Compare the out-of-bag error with the one obtained on the test sample

In [ ]:
# Prediction error on the  test sample
1-rfFit.score(Xr_test,Yb_test)

Cross-validation optimization of the number of variables randomly drawn during the construction of each node.

In [ ]:
param=[{"max_features":list(range(2,10,1))}]
rf= GridSearchCV(RandomForestClassifier(n_estimators=100),
        param,cv=5,n_jobs=-1)
rfOpt=rf.fit(Xr_train, Yb_train)
# paramètre optimal
print("Best score = %f, Best parameter = %s" % (1. - rfOpt.best_score_,rfOpt.best_params_))

Several runs, randomized by cross-validation, can lead to different "optimal" values for this parameter without compromising the quality of the prediction on the test sample.

In [ ]:
# Prediction error on the  test sample
1-rfOpt.score(Xr_test,Yb_test)

In [ ]:
# # Prediction
y_chap = rfFit.predict(Xr_test)
# Confusion matrix
table=pd.crosstab(y_chap,Yb_test)
print(table)

**Exercise** Test different values of min_samples_split from the optimal one. Conclusion on the sensitivity of optimizing this parameter?


As with R, it is possible to calculate a variable importance indicator to help with some form of interpretation. This depends on the position of the variable in the tree and therefore corresponds to R's mean decrease in Gini index rather than the mean descrease in accuracy. The forest must be re-estimated, as GridSearch does not know the importance parameter.

In [ ]:
rf= RandomForestClassifier(n_estimators=100,max_features=2)
rfFit=rf.fit(Xr_train, Yb_train)
# Decreasing importance of variables
importances = rfFit.feature_importances_
indices = np.argsort(importances)[::-1]
for f in range(Xr_train.shape[1]):
    print(dfC.columns[indices[f]], importances[indices[f]])

In [ ]:
# Graph of importances
plt.figure()
plt.title("Importance des variables")
plt.bar(range(Xr_train.shape[1]), importances[indices]);
plt.xticks(range(Xr_train.shape[1]), indices);
plt.xlim([-1, Xr_train.shape[1]]);
plt.show()

**Question** Compare the variable importances with the selections made previously.

**Exercise** Then replace the `RandomForestClassifier` function with the `RandomForestRegressor` regression function. Optimize the parameter, calculate the forecast, residuals and MSE.

# <FONT COLOR="Red">Episode 3 : Neural Networks </font>

Neural networks (multilayer perceptron) are only included in the `Scikit-learn` package from version 0.18. Deep learning" methods require the installation of the [*theano*](http://deeplearning.net/software/theano/) and [*Lasagne*](http://lasagne.readthedocs.io/en/latest/index.html) libraries or [*theano*](http://deeplearning.net/software/theano/), [*TensorFlow*](https://www.tensorflow.org/versions/r0.11/get_started/os_setup.html) and [*Keras*](https://keras.io/) libraries. The latter are considerably more complex to install, especially under Windows. They will be the subject of another tutorial.

In [ ]:
from sklearn.neural_network import MLPClassifier

Definition of parameters, including the number of neurons and `alpha`, which sets the default regularization of 10-5. The number of neurons is optimized, but it can be `alpha` with a large number of neurons. The default maximum number of iterations (200) seems insufficient. It is set to 500.

In [ ]:
param_grid=[{"hidden_layer_sizes":list([(5,),(6,),(7,),(8,)])}]
nnet= GridSearchCV(MLPClassifier(max_iter=500),param_grid,cv=10,n_jobs=-1)
nnetOpt=nnet.fit(Xr_train, Yb_train)
# optimal parameter
print("Best score = %f, Best parameter = %s" % (1. - nnetOpt.best_score_,nnetOpt.best_params_))

In [ ]:
#  Prediction error on the  test sampl
1-nnetOpt.score(Xr_test,Yb_test)

In [ ]:
# # Prediction on the  test sample
y_chap = nnetOpt.predict(Xr_test)
# confusion matrix
table=pd.crosstab(y_chap,Yb_test)
print(table)

**Exercise** Replace the MLPClassifier function with the [MLPRegressor](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html) regression function. Optimize the parameter, calculate the forecast, residuals and MSE.

## Synthesis : comparison of the algorithms  </font>

### ROC curves

In any method, the prediction of whether or not a test will be exceeded is associated with the choice of a threshold, which by default is 0.5. Optimizing this threshold depends on the respective costs associated with false positives and false negatives, which are not necessarily equal. The ROC curve shows the influence of this threshold on the false positive and true positive rates.  

In [ ]:
from sklearn.metrics import roc_curve
listMethod=[["RF",rfOpt],["NN",nnetOpt],["Tree",treeOpt],["K-nn",knnOpt],["Logit",logitOpt]]

In [ ]:
for method in enumerate(listMethod):
    probas_ = method[1][1].fit(Xr_train, Yb_train).predict_proba(Xr_test)
    fpr, tpr, thresholds = roc_curve(Yb_test, probas_[:,1])
    plt.plot(fpr, tpr, lw=1,label="%s"%method[1][0])
plt.xlabel('Taux de faux positifs')
plt.ylabel('Taux de vrais positifs')
plt.legend(loc="best")
plt.show()

**Question**  Can the AUC (area under the curve) criterion be used to order the curves and therefore the methods? 

 Rather, it's at an acceptable false-positive rate, and therefore at a fixed threshold value, that we need to choose the preferred learning method. 

# <FONT COLOR="Red">Episode 5 :  Industrialization of learning </font>

### Iteration on several  test samples :*Monte Carlo* cross-validation 

he test sample is small, so the forecast error estimate may have a large variance. This is reduced by performing a form of cross-validation (*Monte Carlo*), drawing several pairs of training and test samples to iterate the previous treatments. Data are normalized for all methods.

Scikit-learn's functionality lends itself well to automating these processes, which involve extracting samples, estimating several models, optimizing their parameters and estimating the prediction error on the test.

The code is compact and efficient to run, as it is well parallelized by the functions used.

In [ ]:
from sklearn.utils import check_random_state
import time
check_random_state(13)
tps0=time.perf_counter()
# definition of the estimators
logit= LogisticRegression(penalty="l1",solver="liblinear")
knn  = KNeighborsClassifier()
tree = DecisionTreeClassifier()
nnet = MLPClassifier(max_iter=600)
rf   = RandomForestClassifier(n_estimators=100)
svm  = SVC()
# Number of itérations
B=3 # Too small value  ! To execute after the  test, it is better to take at least B=30
# definition  of the  parameters
listMethGrid=[[svm,{"C":[0.4,0.5,0.6,0.8,1,1.4]}],
    [rf,{"max_features":list(range(2,10,2))}],
    [nnet,{"hidden_layer_sizes":list([(5,),(6,),(7,),(8,)])}],
    [tree,{"max_depth":list(range(2,10))}],
    [knn,{"n_neighbors":list(range(1,15))}],
    [logit,{"C":[0.5,1,5,10,12,15,30]}]]
# Initialization at 0 of the errors for each method (column)  and each iteration (line)
arrayErreur=np.empty((B,6))
for i in range(B):   # iterations on  B test samples
    # extraction  of learning and test samples
    X_train,X_test,Yb_train,Yb_test=train_test_split(dfC,Yb,test_size=200)
    scaler = StandardScaler()  
    scaler.fit(X_train)  
    X_train = scaler.transform(X_train)  
    # Same transformation for the  test
    X_test = scaler.transform(X_test)
    # optimisation  of each method  and computation of the test error
    for j,(method, grid_list) in enumerate(listMethGrid):
        methodGrid=GridSearchCV(method,grid_list,cv=10,n_jobs=-1).fit(X_train, Yb_train)
        methodOpt = methodGrid.best_estimator_
        methFit=methodOpt.fit(X_train, Yb_train)
        arrayErreur[i,j]=1-methFit.score(X_test,Yb_test)
tps1=time.perf_counter()
print("Temps execution en mn :",(tps1 - tps0))
dataframeErreur=pd.DataFrame(arrayErreur,columns=["SVM","RF","NN","Tree","Knn","Logit"])    

In [ ]:
# Distribution of the prediction errors
#.
dataframeErreur[["SVM","RF","NN","Tree","Knn","Logit"]].boxplot(return_type='dict')
plt.show()

In [ ]:
# Means
dataframeErreur.mean()

### Conclusion 
**Question** Which method(s) would you retain?

This example, processed in R and then in Python, sums up the interest and context of learning methods.
* Compared with the *base line*: MOCAGE prediction with an average error rate of 30%, an elementary statistical model significantly improves the result.
* A more sophisticated method, in this case *SVM* or *random forest*, provides a statistically significant improvement, but only at the cost of the fine-tuned interpretation of results provided by logistic regression.
* Python, a machine learning tool, is more efficient than R for simulations.
* On the other hand, R, a "statistical learning" tool, enables the selection and interpretation of variables and their **interactions** for a classic linear or logistic regression model. Taking interactions into account (quadratic model) significantly improves forecast quality.
* Random forests and SVMs do better on this example, as is often the case with *boosting*, but other examples highlight other methods: neurons for physical modeling, SVMs for virtual screening of molecules, PLS regression for near infrared spectrometry (NIR)... no general rule.
* Jupyter is an effective teaching aid for analysis without extensive code development.
* Before moving on to [Julia](http://julialang.org/), R and Python are highly complementary.
